In [1]:
from pandas import DataFrame
from pandas import Series
from pandas import concat
from pandas import read_csv
from pandas import datetime
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential, Model
from keras.layers import Dense, Input
from keras.layers import LSTM
from math import sqrt
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
import numpy as np

# frame a sequence as a supervised learning problem
def timeseries_to_supervised(data, lag=1):
    df = DataFrame(data)
    columns = [df.shift(i) for i in range(1, lag+1)]
    columns.append(df)
    df = concat(columns, axis=1)
    df.fillna(0, inplace=True)
    return df

def create_X_train(data, m = 365):
    X = np.zeros((len(data), m))
    for i in range(m):
        df = timeseries_to_supervised(data, i).values
        for j in range(len(df)):
            X[j,i] = df[j,0]
    return X

# create a differenced series
#for stationarity
def difference(dataset, interval=1):
    diff = list()
    for i in range(interval, len(dataset)):
        value = dataset[i] - dataset[i - interval]
        diff.append(value)
    return Series(diff)

# invert differenced value
def inverse_difference(history, yhat, interval=1):
    return yhat + history[-interval]

def normalize(X):
    mu = np.mean(X)
    std = np.std(X)
    norm_X = (X-mu)/std
    return norm_X, mu, std

# load dataset
series = read_csv('RF_kdm.csv')
series = series['RAINFALL(mm)']
# transform data to be stationary
series = series.dropna()
raw_values = series.values
print(len(raw_values))
print(raw_values)
diff_values = difference(raw_values, 1)

# transform data to be supervised learning
X = create_X_train(diff_values)
y = diff_values
X, mu, std = normalize(X)
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.1)

inp = Input(shape = (1, X_train.shape[1]))
lstm_layer = LSTM(4)(inp)
dense = Dense(1)(lstm_layer)
model = Model(inputs = inp, outputs = dense)
X_train2 = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
model.compile(loss = 'mse', optimizer = 'adam')
model.fit(X_train2, y_train, batch_size = 32, epochs = 5000)

# walk-forward validation on the test data
predictions = list()
X_test1 = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))
yhat = model.predict(X_test1)
for (i,y) in enumerate(yhat):
    y = y[0]
    y = y*std + mu
    predictions.append(y)

# report performance
predictions = [p/10 for p in predictions]
rmse = sqrt(mean_squared_error(y_test, predictions))
print('Test RMSE: %.3f' % rmse)
# line plot of observed vs predicted
plt.figure(1)
plt.stem(y_test)
#pyplot.plot(predictions)
plt.show()
plt.figure(2)
plt.stem(predictions)
plt.show()

C:\Users\RAG N PATEL\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


758
[  0.   1.   2.   0.   0.   0.   3.   0.   0.   0.   0.   0.   0.   0.
   1.   0.   1.   4.   1.   0.   0.   0.   0.   0.   0.   2.   0.   0.
   0.   0.   0.   0.   6.   5.   5.   2.   2.   0.   0.   0.   1.   0.
   0.   3.   0.   3.   1.   0.   3.   0.   0.   0.   3.   4.   1.   2.
   1.   1.   0.   1.   4.   3.   0.   3.   1.   1.   1.   1.   1.   1.
   1.   1.   0.   1.   0.   0.   1.   0.   0.   1.   0.   1.   0.   0.
  16.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   3.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   1.   0.   0.   0.   0.
   0.   0.   1.   2.   0.  23.  15.  95.   4.   0.   0.   3.   0.  20.
  15.   2.   0.   0.   0.   0.   1.  27.  32.   1.   0.   0.   0.   0.
 103.   8.   0.   0.   1.   0.   0.   0.  45. 232.  88.   9.  47.  75.
  14.  49.   2.   0.   4.   0.   8.   1.  31.   0.   0.   0.  18.  20.
   0.  10.   0.   0.   1.   7.   1.  68.  14.   6.  21.  12.   1.   0.
  

681/681 [==============================] - 0s 122us/step - loss: 220.5114
Epoch 138/5000
681/681 [==============================] - 0s 123us/step - loss: 220.4831
Epoch 139/5000
681/681 [==============================] - 0s 95us/step - loss: 219.9715
Epoch 140/5000
681/681 [==============================] - 0s 105us/step - loss: 219.3866
Epoch 141/5000
681/681 [==============================] - 0s 104us/step - loss: 219.2421
Epoch 142/5000
681/681 [==============================] - 0s 110us/step - loss: 220.1941
Epoch 143/5000
681/681 [==============================] - 0s 107us/step - loss: 219.2147
Epoch 144/5000
681/681 [==============================] - 0s 92us/step - loss: 218.2548
Epoch 145/5000
681/681 [==============================] - 0s 132us/step - loss: 218.8184
Epoch 146/5000
681/681 [==============================] - 0s 95us/step - loss: 221.0780
Epoch 147/5000
681/681 [==============================] - 0s 129us/step - loss: 218.6561
Epoch 148/5000
681/681 [===============

681/681 [==============================] - 0s 75us/step - loss: 172.7774
Epoch 323/5000
681/681 [==============================] - 0s 117us/step - loss: 172.6242
Epoch 324/5000
681/681 [==============================] - 0s 130us/step - loss: 173.4446
Epoch 325/5000
681/681 [==============================] - 0s 89us/step - loss: 176.8942
Epoch 326/5000
681/681 [==============================] - 0s 85us/step - loss: 177.3178
Epoch 327/5000
681/681 [==============================] - 0s 88us/step - loss: 175.7467
Epoch 328/5000
681/681 [==============================] - 0s 81us/step - loss: 174.1524
Epoch 329/5000
681/681 [==============================] - 0s 132us/step - loss: 173.1819
Epoch 330/5000
681/681 [==============================] - 0s 75us/step - loss: 171.7795
Epoch 331/5000
681/681 [==============================] - 0s 92us/step - loss: 170.4244
Epoch 332/5000
681/681 [==============================] - 0s 100us/step - loss: 169.2583
Epoch 333/5000
681/681 [===================

681/681 [==============================] - 0s 89us/step - loss: 139.6518
Epoch 507/5000
681/681 [==============================] - 0s 94us/step - loss: 138.2879
Epoch 508/5000
681/681 [==============================] - 0s 79us/step - loss: 137.9054
Epoch 509/5000
681/681 [==============================] - 0s 81us/step - loss: 137.7470
Epoch 510/5000
681/681 [==============================] - 0s 82us/step - loss: 137.2580
Epoch 511/5000
681/681 [==============================] - 0s 92us/step - loss: 136.7157
Epoch 512/5000
681/681 [==============================] - 0s 78us/step - loss: 137.1596
Epoch 513/5000
681/681 [==============================] - 0s 79us/step - loss: 138.7497
Epoch 514/5000
681/681 [==============================] - 0s 88us/step - loss: 139.1177
Epoch 515/5000
681/681 [==============================] - 0s 78us/step - loss: 137.5983
Epoch 516/5000
681/681 [==============================] - 0s 75us/step - loss: 136.9600
Epoch 517/5000
681/681 [=======================

...

681/681 [==============================] - 0s 91us/step - loss: 148.7936
Epoch 4673/5000
681/681 [==============================] - 0s 75us/step - loss: 173.2347
Epoch 4674/5000
681/681 [==============================] - 0s 81us/step - loss: 140.4033
Epoch 4675/5000
681/681 [==============================] - 0s 76us/step - loss: 75.3808
Epoch 4676/5000
681/681 [==============================] - 0s 88us/step - loss: 64.4219
Epoch 4677/5000
681/681 [==============================] - 0s 72us/step - loss: 60.4639
Epoch 4678/5000
681/681 [==============================] - 0s 70us/step - loss: 59.3935
Epoch 4679/5000
681/681 [==============================] - 0s 70us/step - loss: 58.7285
Epoch 4680/5000
681/681 [==============================] - 0s 78us/step - loss: 59.3648
Epoch 4681/5000
681/681 [==============================] - 0s 72us/step - loss: 58.7349
Epoch 4682/5000
681/681 [==============================] - 0s 70us/step - loss: 58.7999
Epoch 4683/5000
681/681 [====================

681/681 [==============================] - 0s 73us/step - loss: 49.3565
Epoch 4859/5000
681/681 [==============================] - 0s 76us/step - loss: 49.1852
Epoch 4860/5000
681/681 [==============================] - 0s 81us/step - loss: 57.1017
Epoch 4861/5000
681/681 [==============================] - 0s 76us/step - loss: 50.1371
Epoch 4862/5000
681/681 [==============================] - 0s 83us/step - loss: 49.8008
Epoch 4863/5000
681/681 [==============================] - 0s 111us/step - loss: 49.9535
Epoch 4864/5000
681/681 [==============================] - 0s 103us/step - loss: 49.6242
Epoch 4865/5000
681/681 [==============================] - 0s 88us/step - loss: 51.1636
Epoch 4866/5000
681/681 [==============================] - 0s 81us/step - loss: 75.2277
Epoch 4867/5000
681/681 [==============================] - 0s 105us/step - loss: 95.7026
Epoch 4868/5000
681/681 [==============================] - 0s 78us/step - loss: 59.9077
Epoch 4869/5000
681/681 [====================

Epoch 4951/5000
681/681 [==============================] - 0s 89us/step - loss: 112.6406
Epoch 4952/5000
681/681 [==============================] - 0s 94us/step - loss: 112.2929
Epoch 4953/5000
681/681 [==============================] - 0s 86us/step - loss: 112.0862
Epoch 4954/5000
681/681 [==============================] - 0s 95us/step - loss: 112.0462
Epoch 4955/5000
681/681 [==============================] - 0s 95us/step - loss: 111.8988
Epoch 4956/5000
681/681 [==============================] - 0s 82us/step - loss: 112.0587
Epoch 4957/5000
681/681 [==============================] - 0s 73us/step - loss: 111.1886
Epoch 4958/5000
681/681 [==============================] - 0s 81us/step - loss: 110.2111
Epoch 4959/5000
681/681 [==============================] - 0s 123us/step - loss: 109.1342
Epoch 4960/5000
681/681 [==============================] - 0s 111us/step - loss: 108.0220
Epoch 4961/5000
681/681 [==============================] - 0s 79us/step - loss: 107.5297
Epoch 4962/5000
681

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

In [3]:
len(predictions)

76

In [4]:
predictions

[0.04251753310767928,
 -0.02537898657312605,
 0.04251753310767928,
 -0.1607858731524056,
 0.04251753310767928,
 -21.473085873261304,
 0.04251753310767928,
 0.04251753310767928,
 0.04251753310767928,
 0.04251753310767928,
 0.04251753310767928,
 -0.008379213694723043,
 0.04251753310767928,
 0.04251753310767928,
 -0.09907986526334528,
 0.04251753310767928,
 0.04251753310767928,
 0.04251753310767928,
 0.04251753310767928,
 0.952718859703969,
 0.04251753310767928,
 0.04251753310767928,
 -0.6280277229543895,
 0.04251753310767928,
 -37.53841276229299,
 2.343604979837143,
 0.04251753310767928,
 0.04251753310767928,
 -0.08069838946066392,
 0.04251753310767928,
 0.04251753310767928,
 0.04251753310767928,
 0.12783618027256222,
 -0.008379213694723043,
 0.04251753310767928,
 0.04251753310767928,
 -0.02537898657312605,
 0.04251753310767928,
 0.04251753310767928,
 0.3169876904239358,
 -0.7313839286001902,
 0.06044648611856642,
 0.04251753310767928,
 0.04251753310767928,
 1.716954434579114,
 0.0425175